In [170]:
from nltk.corpus import stopwords
import nltk
from nltk import word_tokenize, FreqDist
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from gensim.models import Word2Vec
import pandas as pd

In [171]:
text = pd.read_csv('combined_text_data.csv')
stock_orignial = pd.read_csv('stock_data.csv')
stock = pd.read_csv('interpolated_stock_data.csv')

In [172]:
# stock = stock.iloc[::-1]

In [173]:
stock.rename(columns={'Unnamed: 0':'Date'}, inplace=True)

In [174]:
stock.shape

(1776, 2)

In [175]:
stock.head()

,Date,signal
0,2012-11-19,up
1,2012-11-20,stay
2,2012-11-21,stay
3,2012-11-22,down
4,2012-11-23,down


In [176]:
text.shape

(3205, 3859)

In [177]:
text.columns

Index(['Time', 'retweet_from', 'tweet_length', 'encoded_sentiment',
       'sentiment', 'polarity', 'tweet', 'ability', 'able', 'aboard',
       ...
       'you', 'your', 'yours', 'yourself', 'yr', 'yup', 'zero', 'zip', 'zone',
       'zoo'],
      dtype='object', length=3859)

In [178]:
text.head()

,Time,retweet_from,tweet_length,encoded_sentiment,sentiment,polarity,tweet,ability,able,aboard,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,2017-09-29,NaN,105,1,positive,0.366667,"Assuming max acceleration of 2 to 3 g's, but i...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-09-29,SpaceX,113,1,positive,0.200000,BFR is capable of transporting satellites to o...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-09-29,NaN,6,0,neutral,0.000000,Yup :),0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2017-09-29,NaN,7,0,neutral,0.000000,Part 2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-09-29,NaN,96,1,positive,0.650000,Fly to most places on Earth in under 30 mins a...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [179]:
stock.set_index('Date', inplace=True)

In [180]:
idx = pd.date_range('2012-11-19', '2017-09-29')

# s = pd.Series({'09-02-2013': 2,
#                '09-03-2013': 10,
#                '09-06-2013': 5,
#                '09-07-2013': 1})

# s.index = pd.DatetimeIndex(s.index)
# s = s.reindex(idx, fill_value='NaN')
# print(s)

In [181]:
stock.index = pd.DatetimeIndex(stock.index)

In [182]:
stock = stock.reindex(idx, fill_value='NaN')

In [196]:
# stock.index.name = 'Date'

---

In [189]:
text.rename(columns={'Time':'Date'}, inplace=True)

In [195]:
# text.set_index('Date')

In [191]:
stock.head()

,signal
Date,
2012-11-19,up
2012-11-20,stay
2012-11-21,stay
2012-11-22,down
2012-11-23,down


In [198]:
# take_2 = pd.merge(text, stock, on='Date', how='left')

In [193]:
take_2.shape

(3205, 3860)

In [194]:
take_2.head(10)

,Date,retweet_from,tweet_length,encoded_sentiment,sentiment,polarity,tweet,ability,able,aboard,...,your,yours,yourself,yr,yup,zero,zip,zone,zoo,signal_y
0,2017-09-29,NaN,105,1,positive,0.366667,"Assuming max acceleration of 2 to 3 g's, but i...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
1,2017-09-29,SpaceX,113,1,positive,0.200000,BFR is capable of transporting satellites to o...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
2,2017-09-29,NaN,6,0,neutral,0.000000,Yup :),0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,stay
3,2017-09-29,NaN,7,0,neutral,0.000000,Part 2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
4,2017-09-29,NaN,96,1,positive,0.650000,Fly to most places on Earth in under 30 mins a...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
5,2017-09-29,SpaceX,80,1,positive,0.125000,"Supporting the creation of a permanent, self-s...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
6,2017-09-29,NaN,57,-1,negative,-0.166667,BFR will take you anywhere on Earth in less th...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
7,2017-09-29,NaN,84,0,neutral,0.000000,Mars City Opposite of Earth. Dawn and dusk sky...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
8,2017-09-29,NaN,16,-1,negative,-0.800000,Moon Base Alpha,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
9,2017-09-29,NaN,60,1,positive,0.357143,Will be announcing something really special at...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay


In [ ]:
take_2.signal_y.value_counts()

---

#### PCAAAAAAAA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=38)

In [ ]:
tfidf_df = take_2.iloc[:, 7:-1]

In [ ]:
tfidf_df.head()

In [ ]:
pca.fit(tfidf_df)

In [ ]:
x_pca = pca.transform(tfidf_df)
x_pca

In [ ]:
x_pca.shape

In [ ]:
pca_df = pd.DataFrame(x_pca)

In [ ]:
pca_df.head()

In [ ]:
# pd.get_dummies(stock)

In [ ]:
labels = []

for i in range(1, 39):
    labels.append('pc_' + str(i))

In [ ]:
# labels

In [ ]:
pca_df.columns = labels

In [ ]:
df_with_tfidf_pca = pd.concat([take_2.iloc[:, :7], pca_df], axis=1)

In [ ]:
df_with_tfidf_pca

In [ ]:
df_with_tfidf_pca['retweet_from'] = df_with_tfidf_pca['retweet_from'].fillna('not')

In [ ]:
df_with_tfidf_pca['retweet_from'] = le.fit_transform(df_with_tfidf_pca['retweet_from'])

In [ ]:
df_with_tfidf_pca.drop(columns = ['sentiment', 'Date'], inplace= True)

In [ ]:
df_with_tfidf_pca.info()

In [ ]:
# take_23 = pd.merge(stock, text, on ='Date')   
# 2012-11-19   2017-09-29

In [ ]:
# this is without weekends
# take_23.shape

In [ ]:
# take_23

In [ ]:
# take_23['retweet_from'] = take_23['retweet_from'].fillna('not')

In [ ]:
# take_23.shape

In [ ]:
# take_23.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
take_23['retweet_from'] = le.fit_transform(take_23['retweet_from'])

In [ ]:
take_23.drop(columns = ['sentiment'], inplace= True)

In [ ]:
#Same but with no date
take_23.drop(columns = ['Date'], inplace= True)

#### fixing the tweet column to match cleaned tweets

In [ ]:
tweets = df_with_tfidf_pca['tweet'].tolist()

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
english = set(nltk.corpus.words.words()+ ['yup'])

In [ ]:
def lemmadata(doc):
        pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
        raw_tokens = nltk.regexp_tokenize(doc, pattern)
        tokens = [i.lower() for i in raw_tokens]
        lemmatized = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
        words = list(filter(lambda w: w in english, lemmatized))
        return words

In [ ]:
lemma = [lemmadata(post) for post in tweets]

In [ ]:
process_lems = list(map(lemmadata, df_with_tfidf_pca['tweet']))

In [ ]:
# process_lems[:3]

In [ ]:
clean_tweets_lem = []

for i in range(0, len(process_lems)):
    clean = ' '.join(process_lems[i])
    clean_tweets_lem.append(clean)

In [ ]:
# df['B'] = df1['E'].values

In [ ]:
clean_tweet_df = pd.DataFrame(clean_tweets_lem)

In [ ]:
# clean_tweet_df.head()

In [ ]:
clean_tweet_df[0][0]

In [ ]:
df_with_tfidf_pca['tweet'] = clean_tweet_df[0].values

In [ ]:
# we added signal to the dataframe
df_with_tfidf_pca = pd.concat([df_with_tfidf_pca, take_2.iloc[:,-1]], axis=1)

In [197]:
df_with_tfidf_pca.head()

,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,pc_1,pc_2,pc_3,pc_4,pc_5,...,pc_30,pc_31,pc_32,pc_33,pc_34,pc_35,pc_36,pc_37,pc_38,signal_y
0,171,105,1,0.366667,assuming acceleration of to but in a comfortab...,-0.031750,-0.028850,-0.030290,-0.018736,-0.060675,...,0.017885,0.045822,-0.006862,-0.012954,-0.005468,-0.100572,-0.029374,0.041363,0.035932,stay
1,77,113,1,0.200000,is capable of transporting satellite to orbit ...,-0.059331,-0.099826,-0.154772,-0.040355,0.072285,...,-0.011667,0.003882,0.018002,-0.019813,-0.035809,-0.034079,-0.014687,-0.008986,-0.019845,stay
2,171,6,0,0.000000,yup,-0.002330,0.033983,0.069531,-0.014177,-0.037774,...,0.015722,-0.025726,0.065735,0.041686,-0.021751,-0.014138,-0.037047,-0.050031,-0.084722,stay
3,171,7,0,0.000000,part,-0.005050,0.024117,0.043362,-0.001804,-0.016279,...,-0.009052,-0.008601,0.001285,-0.007619,-0.005289,-0.025700,-0.008989,-0.011967,-0.004665,stay
4,171,96,1,0.650000,fly to most place on earth in under min and an...,-0.031167,-0.027725,-0.012331,-0.039909,-0.070462,...,0.020042,0.030526,-0.039462,0.088186,-0.012277,0.033849,0.040576,0.051975,-0.031236,stay


In [199]:
df_with_tfidf_pca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3205 entries, 0 to 3204
Data columns (total 44 columns):
retweet_from         3205 non-null int64
tweet_length         3205 non-null int64
encoded_sentiment    3205 non-null int64
polarity             3205 non-null float64
tweet                3205 non-null object
pc_1                 3205 non-null float64
pc_2                 3205 non-null float64
pc_3                 3205 non-null float64
pc_4                 3205 non-null float64
pc_5                 3205 non-null float64
pc_6                 3205 non-null float64
pc_7                 3205 non-null float64
pc_8                 3205 non-null float64
pc_9                 3205 non-null float64
pc_10                3205 non-null float64
pc_11                3205 non-null float64
pc_12                3205 non-null float64
pc_13                3205 non-null float64
pc_14                3205 non-null float64
pc_15                3205 non-null float64
pc_16                3205 non-null float

In [ ]:
# take_23.to_csv(r'/Users/sproul/Desktop/ds-projects/project_mod4_AAR/CLEAN_EDIT_model_data.csv')

In [200]:
df_with_tfidf_pca.to_csv(r'/Users/sproul/Desktop/ds-projects/project_mod4_AAR/updated_merged_data.csv')